In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df_sample = pd.read_csv("data-files/sample_submission.csv")
df_test = pd.read_csv("data-files/test.csv")
df_train = pd.read_csv("data-files/train.csv")

In [3]:
df_train.columns

Index(['ID', '설립연도', '국가', '분야', '투자단계', '직원 수', '인수여부', '상장여부', '고객수(백만명)',
       '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)', '성공확률'],
      dtype='object')

In [4]:
# ID 제거
df_train = df_train.drop(columns=["ID"])
df_test = df_test.drop(columns=["ID"])

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   설립연도            4376 non-null   int64  
 1   국가              4376 non-null   object 
 2   분야              3519 non-null   object 
 3   투자단계            4376 non-null   object 
 4   직원 수            4202 non-null   float64
 5   인수여부            4376 non-null   object 
 6   상장여부            4376 non-null   object 
 7   고객수(백만명)        3056 non-null   float64
 8   총 투자금(억원)       4376 non-null   float64
 9   연매출(억원)         4376 non-null   float64
 10  SNS 팔로워 수(백만명)  4376 non-null   float64
 11  기업가치(백억원)       3156 non-null   object 
 12  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 444.6+ KB


In [6]:
df_train.head()

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8
2,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,0.5
3,2016,CT006,NaN,Seed,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,0.7
4,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,0.1


In [7]:
# 기업가치를 평균값으로 (숫자로 변환)

import re

def parse_valuation(v):
    if pd.isna(v):
        return np.nan
    v = str(v).replace(",","") # 혹시 콤마 있으면 제거
    v = ''.join(filter(lambda x: x.isdigit() or x == '-', v)) # 숫자와 '-'만 남긴다

    if '-' in v:
        low, high = v.split('-')
        return (float(low) + float(high)) / 2
    else:
        return float(v)
    
df_train['기업가치(백억원)'] = df_train['기업가치(백억원)'].apply(parse_valuation)
df_test['기업가치(백억원)'] = df_test['기업가치(백억원)'].apply(parse_valuation)

In [8]:
# 결측치 처리하기

# 중앙값으로
for col in ['직원 수', '고객수(백만명)', '기업가치(백억원)']:
    median = df_train[col].median()
    df_train[col] = df_train[col].fillna(median)
    df_test[col] = df_test[col].fillna(median)

# 분야는 Unknown으로
df_train['분야'] = df_train['분야'].fillna('Unknown')
df_test['분야'] = df_test['분야'].fillna('Unknown')

In [9]:
# feature 가공

# 설립나이로 바꾸기
Current_year = 2025
df_train['설립나이'] = Current_year - df_train['설립연도']
df_test['설립나이'] = Current_year - df_test['설립연도']

In [10]:
# 범주형 데이터 인코딩

# 투자단계
le_invest = LabelEncoder()
df_train['투자단계'] = le_invest.fit_transform(df_train['투자단계'])
df_test['투자단계'] = le_invest.fit_transform(df_test['투자단계'])

# 인수여부 / 상장여부는 0 또는 1로 변환 (성공 1, 실패 0)
for col in ['인수여부', '상장여부']:
    df_train[col] = df_train[col].map({'Yes':1, 'No':0})
    df_test[col] = df_test[col].map({'Yes':1, 'No':0})

In [11]:
# 국가, 분야도 Label Encoding으로 일단
le_country = LabelEncoder()
df_train['국가'] = le_country.fit_transform(df_train['국가'])
df_test['국가'] = le_country.fit_transform(df_test['국가'])

le_field = LabelEncoder()
df_train['분야'] = le_field.fit_transform(df_train['분야'])
df_test['분야'] = le_field.fit_transform(df_test['분야'])

In [12]:
# 사용할 feature들 정리

features = [
    '설립나이', '국가', '분야', '투자단계',
    '직원 수', '인수여부', '상장여부', '고객수(백만명)',
    '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)'
]

X = df_train[features]
y = df_train['성공확률']
X_test = df_test[features]

In [13]:
# 숫자형 feature 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)


In [14]:
# 데이터 나누기 (train용 / validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# 모델 정의(MLP) - Multi-Layer Perceptron
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(features), 64) # 첫 번째 은닉층
        self.fc2 = nn.Linear(64, 32) # 첫 번째 -> 두 번째 은닉층. 노드 32로 줄임
        self.fc3 = nn.Linear(32, 1) # 두 번째 -> 출력층. 노드 1로(성공확률/정답) 변환

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleMLP()

In [26]:
# 학습 준비
criterion = nn.BCEWithLogitsLoss() # Binary Corss Entropy Loss (이진 분류 확률 예측)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Tensor 변환
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)

In [27]:
# 학습
epochs = 500
for epoch in range(epochs):
    model.train() # 모델을 학습 모드로
    optimizer.zero_grad() # 이전에 쌓인 gradient 초기화(매번 0으로)
    output = model(X_train_tensor) # 학습데이터를 모델에 넣어 예측
    print(type(output))
    loss = criterion(output, y_train_tensor) # 예측과 정답 사이의 Loss 계산
    loss.backward() # Loss함수를 기준으로 각 파라미터에 대한 gradient 게산
    optimizer.step() # 계산된 gradient를 이용해 파라미터 업데이트

    if epoch % 50 == 0:
        model.eval()
        val_output = model(X_val_tensor)
        val_loss = criterion(val_output, y_val_tensor)
        print(f"Epoch {epoch}, Train Loss: {loss.item()}, Val Loss: {val_loss.item()}")

<class 'torch.Tensor'>
Epoch 0, Train Loss: 0.6956114768981934, Val Loss: 0.6948807835578918
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'to

In [28]:
print('criterion type:', type(criterion))
print('output type:', type(output))
print('y_train_tensor type:', type(y_train_tensor))

criterion type: <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>
output type: <class 'torch.Tensor'>
y_train_tensor type: <class 'torch.Tensor'>


In [29]:
# ===== 14. 예측 및 저장 ===== #
model.eval()
preds = model(X_test_tensor).detach().numpy().flatten()

# 결과 저장
df_sample['성공확률'] = preds
df_sample.to_csv('submission.csv', index=False)